In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import sys
import math
sys.path.append("..") 
from scipy import stats
from utils import * 

<h3>广东

In [ ]:
region='GD'
variate=['pop','third','area']
countydf=pd.read_csv('../Data/'+region+'/'+region+'_county_flow_detail.csv')
citydf=pd.read_csv('../Data/'+region+'/'+region+'_city_flow_detail.csv')
county_city_rela=load_relation('../Data/'+region+'/city_county_rela.csv')

variatelist=['dist_l']
for v in variate:
    variatelist.append('o_'+v+'_l')
    variatelist.append('d_'+v+'_l')
print(variatelist)
formula = "{}~{}".format("flow_intensity_l", "+".join(variatelist))
print(formula)

cityres = smf.ols(formula = formula, data = citydf).fit()     
print(cityres.summary())
countyres=smf.ols(formula = formula, data = countydf).fit() 
print(countyres.summary())
city_fit=np.exp(cityres.predict(citydf))

print("【city拟合】")
evaluate(city_fit,citydf['flow_intensity'])

In [ ]:
county_pred = np.exp(cityres.predict(countydf))
county_fit = np.exp(countyres.predict(countydf))
city_fit=np.exp(cityres.predict(citydf))
countydf['GM_pred']=county_pred
print("【预测county】")
evaluate(county_pred,countydf['flow_intensity'])
print("【预测county+系数调整】")
countydf=finetuning(countydf,county_city_rela,citydf,'GM_pred')
evaluate(countydf['GM_pred_finetune'],countydf['flow_intensity'])